# 错定价信号生成演示

本 Notebook 使用本项目中的 **真实** 信号模块：`compute_mispricing_series` 与 `add_zscore_and_signals`，基于二叉树公平价值与模拟市场价构造错定价序列和 Z-score 入场/离场信号。

数据构造与 `examples.run_simple_backtest` 一致（GBM 股价 + 带噪声的模拟可转债市场价）。

## 1. 路径与导入

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

cwd = os.getcwd()
project_root = cwd if os.path.isdir(os.path.join(cwd, 'src')) else os.path.abspath(os.path.join(cwd, '..'))
src_dir = os.path.join(project_root, 'src')
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

from cb_arb.params import ConvertibleBondContract, TermStructure, CreditCurve
from cb_arb.signals import (
    compute_mispricing_series,
    add_zscore_and_signals,
    MispricingSignalConfig,
)

# 导入 Notebook 工具函数（用于保存输出文件）
notebooks_dir = os.path.join(project_root, 'notebooks')
if notebooks_dir not in sys.path:
    sys.path.insert(0, notebooks_dir)
from notebook_utils import save_figure, get_figures_dir, get_data_dir

print("项目根目录:", project_root)
print("src 目录已添加到 Python 路径:", src_dir)
print("输出目录:", os.path.join(project_root, 'output'))
print("signals 模块导入成功。")

## 2. 生成与 run_simple_backtest 一致的模拟数据

GBM 股价 + 基于「纯债+折扣转股价值」并加噪声的可转债市场价。

In [ ]:
def simulate_gbm_path(S0, r, q, vol, dates, seed=42):
    dt = 1.0 / 252.0
    n = len(dates)
    rng = np.random.default_rng(seed)
    shocks = rng.normal(0.0, np.sqrt(dt), size=n)
    prices = [S0]
    for eps in shocks[1:]:
        s_prev = prices[-1]
        s_new = s_prev * np.exp((r - q - 0.5 * vol**2) * dt + vol * eps)
        prices.append(s_new)
    return pd.Series(prices, index=dates, name='stock')

dates = pd.date_range("2020-01-01", periods=250, freq="B")
S0, r, q, vol = 100.0, 0.02, 0.01, 0.25
face, conversion_ratio = 100.0, 1.0

stock_series = simulate_gbm_path(S0, r, q, vol, dates)
years = np.linspace(0.0, 1.0, len(dates))
bond_floor = face * np.exp(-0.01 * years)
conv_part = 0.4 * conversion_ratio * stock_series.values
cb_theoretical = bond_floor + conv_part
rng = np.random.default_rng(123)
noise = 0.02 * rng.standard_normal(len(dates))
cb_market = pd.Series(cb_theoretical * (1.0 + noise), index=dates, name='cb_market')

print("stock_series 与 cb_market 长度:", len(stock_series), len(cb_market))
print("索引一致:", stock_series.index.equals(cb_market.index))

## 3. 调用 compute_mispricing_series 得到错定价序列

使用项目内真实的定价与合约参数。

In [ ]:
contract = ConvertibleBondContract(
    face_value=face,
    coupon_rate=0.03,
    maturity=3.0,
    conversion_ratio=conversion_ratio,
    issue_price=100.0,
    call_price=None,
    put_price=None,
    coupon_freq=2,
)
r_curve = TermStructure(rate_fn=lambda t: r)
q_curve = TermStructure(rate_fn=lambda t: q)
credit_curve = CreditCurve(spread_fn=lambda t: 0.03)
steps = 50

df = compute_mispricing_series(
    cb_market_price=cb_market,
    stock_price=stock_series,
    contract=contract,
    r_curve=r_curve,
    q_curve=q_curve,
    credit_curve=credit_curve,
    vol=vol,
    steps=steps,
)

print("列:", list(df.columns))
print(df[['cb_market', 'cb_fair', 'mispricing']].head(10).to_string())

## 4. 添加 Z-score 与入场/离场信号

使用 `add_zscore_and_signals` 与 `MispricingSignalConfig`（与回测示例相同参数）。

In [ ]:
signal_cfg = MispricingSignalConfig(lookback=40, entry_z=-1.5, exit_z=-0.5)
df = add_zscore_and_signals(df, signal_cfg)

print("含 zscore 与 signal 的列:", [c for c in df.columns])
print("signal 取值:", df['signal'].unique())
print("signal=1 的交易日数量:", (df['signal'] == 1).sum())
print(df[['mispricing', 'zscore', 'signal']].tail(15).to_string())

## 5. 错定价、Z-score 与信号可视化（真实数据）

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10, 7), sharex=True)
axes[0].plot(df.index, df['mispricing'], label='mispricing (fair - market)')
axes[0].axhline(0, color='gray', linestyle='--')
axes[0].set_ylabel('错定价')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

axes[1].plot(df.index, df['zscore'], label='zscore', color='C1')
axes[1].axhline(signal_cfg.entry_z, color='green', linestyle='--', label='entry_z')
axes[1].axhline(signal_cfg.exit_z, color='red', linestyle='--', label='exit_z')
axes[1].set_ylabel('Z-score')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

axes[2].plot(df.index, df['signal'], drawstyle='steps-post', label='signal', color='C2')
axes[2].set_ylabel('signal')
axes[2].set_xlabel('日期')
axes[2].legend(loc='upper right')
axes[2].grid(True, alpha=0.3)
plt.suptitle('错定价与信号 (compute_mispricing_series + add_zscore_and_signals 真实输出)')
plt.tight_layout()

# 保存图片
save_figure(fig, '04_signal_generation_mispricing_signals')

# 保存信号数据
from datetime import datetime
data_dir = get_data_dir()
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
csv_path = data_dir / f'signal_data_{timestamp}.csv'
df[['mispricing', 'zscore', 'signal']].to_csv(csv_path, encoding='utf-8-sig')
print(f'信号数据已保存到: {csv_path}')

plt.show()